In [ ]:
import os
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import seaborn as sns

from IPython import display
from tensorflow.keras import layers
from time import strftime
from scipy.signal import spectrogram, stft, istft

In [ ]:
MODEL_NAME = "GanPlayground"
SCALING_FACTOR = 0
BUFFER_SIZE = 1000
BATCH_SIZE = 128
NUM_EXAMPLES_TO_GENERATE = 1
NOISE_DIM = 100
STEAD_PATH_DB_PROCESSED_STFT = "/Users/jarek/git/saigon/data/STEAD-processed-stft.hdf5"

In [ ]:
def plot_single_stream(do, label, fs=100, nperseg=155, file_path=None):
    d0 = pd.DataFrame(data=do)

    fig = plt.figure(figsize=(16, 16), dpi=60)
    ax1 = plt.subplot2grid((4, 1), (0, 0))
    ax2 = plt.subplot2grid((4, 1), (1, 0))
    ax3 = plt.subplot2grid((4, 1), (2, 0), rowspan=2)

    plt.subplots_adjust(hspace=0.5)

    sns.lineplot(data=do, ax=ax1, linewidth=1, legend=None)

    ax1.set_title("Waveform")
    ax1.set(xlabel="Samples", ylabel="Amplitude counts")
    ax1.locator_params(nbins=6, axis="y")

    f, t, Sxx = spectrogram(x=do, fs=fs)

    ax2.clear()
    ax2.set_title("Spectrogram")
    ax2.pcolormesh(t, f, Sxx, shading="gouraud")
    ax2.set(xlabel="Time [sec]", ylabel="Frequency [Hz]")

    f_sftt, t_sftt, Zxx = stft(do, window="hanning", fs=fs, nperseg=nperseg)

    ax3.clear()
    ax3.set_title("STFT")
    ax3.pcolormesh(t_sftt, f_sftt, np.abs(Zxx), shading="auto")

    plt.suptitle(label, fontsize=14)

    if file_path != None:
        plt.savefig(file_path)
    else:
        plt.show()

In [ ]:
def plot_stft(stream, fs=100, nperseg=155):
    f, t, Zxx = stft(stream, window='hanning', fs=fs, nperseg=nperseg)
    # plt.specgram(x_1[0][0], cmap='plasma', Fs=100)
    plt.pcolormesh(t, f, np.abs(Zxx), shading='auto')

In [ ]:
def get_stft_data(file_path, arr_length):
    with h5py.File(file_path, "r") as f:
        keys = f["keys"][:arr_length]
        components = f["components"][:arr_length]
        data = f["data"][:arr_length]
        return (keys, components, data)

# Read processed data

In [ ]:
(keys, components, x_train) = get_stft_data(
    STEAD_PATH_DB_PROCESSED_STFT, 50000
)

# Convert streams to STFT

In [ ]:
# STFT of the stream and then reverse STFT back into original stream
# f, t, Zxx = stft(x_1[1][0], window='hanning', fs=100, nperseg=155)
# k2 = istft(Zxx, window='hanning', fs=100, nperseg=155)

# Scale and reshape data

In [ ]:
SCALING_FACTOR = int(
    max(
        [
            abs(min([x.min() for x in x_train])),
            abs(max([x.max() for x in x_train])),
        ]
    )
)
SCALING_FACTOR

In [ ]:
x_train /= SCALING_FACTOR

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 78, 78, 1)

In [ ]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices(x_train)
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

In [ ]:
train_dataset

# Logs and Tensorboard

In [ ]:
folder_name = f"{MODEL_NAME} at {strftime('%H:%M')}"
log_dir = os.path.join("../log/", folder_name)

try:
    os.makedirs(log_dir)
except OSError as exception:
    print(exception.strerror)
else:
    print("Successfully created dirs!")

# Define GAN

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(3 * 3 * 128, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((3, 3, 128)))

    model.add(
        layers.Conv2DTranspose(
            64, (20, 20), strides=(1, 1), padding="same", use_bias=False
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2DTranspose(
            64, (20, 20), strides=(2, 2), padding="same", use_bias=False
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2DTranspose(
            64, (20, 20), strides=(13, 13), padding="same", use_bias=False
        )
    )
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2DTranspose(
            1,
            (10, 10),
            strides=(1, 1),
            padding="same",
            use_bias=False,
            activation="tanh",
        )
    )
    return model

In [ ]:
generator = make_generator_model()

# noise = tf.random.normal(dtype=tf.dtypes.float32, shape=[78, 78], stddev=5)
noise = tf.random.normal([BATCH_SIZE, NOISE_DIM], stddev=10e5)
generated_stft = generator(noise, training=False)

generated_stft.shape

In [ ]:
generator.summary()

In [ ]:
tf.keras.utils.plot_model(generator, show_shapes=True)

In [ ]:
inversed = istft(generated_stft[0, :, :, 0], window='hanning', fs=100, nperseg=155)
plot_single_stream(inversed[1][:6000], "GAN Generator Noise")

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[78, 78, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_stft)

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([NUM_EXAMPLES_TO_GENERATE, NOISE_DIM])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    for i in range(predictions.shape[0]):
        inversed = istft(
            predictions[i, :, :, 0][:6000], window="hanning", fs=100, nperseg=155
        )
        plot_single_stream(
            inversed[1][:6000],
            f"GAN Event (epoch {epoch})",
            # file_path=f"out/image_at_epoch_{epoch}.jpg"
        )

In [ ]:
train(train_dataset, EPOCHS)